<a href="https://colab.research.google.com/github/bhargavsaidasari/Machine-Learning-Projects/blob/master/Emergency_Vehicle_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U skorch

     |████████████████████████████████| 122kB 8.1MB/s 


In [0]:
import os
import shutil
import hashlib

import numpy as np
import pandas as pd
import cv2

%matplotlib inline
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from PIL import Image
from skorch import NeuralNetClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import albumentations as A
from albumentations.pytorch import ToTensor

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
DIR_IMAGES = "/content/drive/My Drive/train_data/images/"
DIR_DF = "/content/drive/My Drive/train_data/train.csv"

SIZE = 224
BATCH_SIZE = 32

### Check the amount of data for each class


In [0]:
train_data = pd.read_csv('/content/drive/My Drive/train_data/train.csv')
train_data['emergency_or_not'].value_counts()

0    965
1    681
Name: emergency_or_not, dtype: int64

In [0]:
train_data.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


### Create Image metadata

In [0]:
def calculate_hash(im):
    md5 = hashlib.md5()
    md5.update(np.array(im).tostring())
    
    return md5.hexdigest()


def get_image_meta(image_id, image_src, dataset='train'):
    im = Image.open(image_src)
    extrema = im.getextrema()

    meta = {
        'image_id': image_id,
        'dataset': dataset,
        'hash': calculate_hash(im),
        'r_min': extrema[0][0],
        'r_max': extrema[0][1],
        'g_min': extrema[1][0],
        'g_max': extrema[1][1],
        'b_min': extrema[2][0],
        'b_max': extrema[2][1],
        'height': im.size[0],
        'width': im.size[1],
        'format': im.format,
        'mode': im.mode
    }
    return meta

### Dataset Class

In [0]:
class VehicleDataset(Dataset):
  """ Emergency Vehicles Dataset. """
  def __init__(self, csv_file, root_dir, transform=None):
    """ 
    Parameters:
      csv_file(string): Path to the csv file containing the labels.
      root_dir(string): Path to the folder that contains the images.
      transforms(callable): Optional transforms to be applied on a sample."""
    self.vehicles_frame = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform

  def __len__(self):
    return(self.vehicles_frame.shape[0])
  
  def __getitem__(self, idx):
    img_name = self.root_dir + self.vehicles_frame['image_names'].iloc[idx]
    image = cv2.imread(img_name, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    transformed = self.transform(image=image)
    image = transformed['image']
    labels = torch.tensor(self.vehicles_frame['emergency_or_not'].iloc[idx])
    return(image, labels)

### Define Transforms

In [0]:
train_transform = A.Compose([
    A.Resize(height=SIZE, width=SIZE, p=1),
    A.HorizontalFlip(p=0.3),
    A.VerticalFlip(p=0.3),
    A.ShiftScaleRotate(rotate_limit=0.5, p=0.8),

    # Pixels
    A.OneOf([
        A.IAAEmboss(p=1.0),
        A.IAASharpen(p=1.0),
        A.Blur(p=1.0),
    ], p=0.5),

    A.Normalize(p=1.0),
    ToTensor(),
])

transforms_valid = A.Compose([
    A.Resize(height=SIZE, width=SIZE, p=1.0),
    A.Normalize(p=1.0),
    ToTensor(),
])

### Test Dataset Class



In [0]:
train_dataset = VehicleDataset(DIR_DF, DIR_IMAGES, train_transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
images, labels = next(iter(train_dataloader))

###  V1 - Densenet Model

In [0]:
model = models.densenet121(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/checkpoints/densenet121-a639ec97.pth


In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.Softmax(dim=1))
criterion = nn.BCELoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
model.to(device);

In [0]:
epochs = 3
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in train_dataloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.float().to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps[:, -1], labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. ")
            running_loss = 0
            model.train()

Epoch 1/3.. Train loss: 1.342.. 
Epoch 1/3.. Train loss: 0.737.. 
Epoch 1/3.. Train loss: 0.657.. 
Epoch 1/3.. Train loss: 0.597.. 
Epoch 1/3.. Train loss: 0.532.. 
Epoch 1/3.. Train loss: 0.495.. 
Epoch 1/3.. Train loss: 0.467.. 
Epoch 1/3.. Train loss: 0.423.. 
Epoch 1/3.. Train loss: 0.405.. 
Epoch 1/3.. Train loss: 0.377.. 
Epoch 2/3.. Train loss: 0.498.. 
Epoch 2/3.. Train loss: 0.426.. 
Epoch 2/3.. Train loss: 0.408.. 
Epoch 2/3.. Train loss: 0.270.. 
Epoch 2/3.. Train loss: 0.281.. 
Epoch 2/3.. Train loss: 0.270.. 
Epoch 2/3.. Train loss: 0.245.. 
Epoch 2/3.. Train loss: 0.391.. 
Epoch 2/3.. Train loss: 0.401.. 
Epoch 2/3.. Train loss: 0.344.. 
Epoch 3/3.. Train loss: 0.315.. 
Epoch 3/3.. Train loss: 0.289.. 
Epoch 3/3.. Train loss: 0.351.. 
Epoch 3/3.. Train loss: 0.241.. 
Epoch 3/3.. Train loss: 0.310.. 
Epoch 3/3.. Train loss: 0.235.. 
Epoch 3/3.. Train loss: 0.270.. 
Epoch 3/3.. Train loss: 0.202.. 
Epoch 3/3.. Train loss: 0.284.. 
Epoch 3/3.. Train loss: 0.221.. 
Epoch 3/3.

In [0]:
def predict_image(image, transform):
    image_tensor = transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(device)
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    return index

In [0]:
test_df = pd.read_csv('/content/drive/My Drive/train_data/test_vc2kHdQ.csv')
test_df['emergency_or_net'] = 0
test_loader = VehicleDataset('/content/drive/My Drive/train_data/test_vc2kHdQ.csv', DIR_IMAGES, transforms_valid)

In [0]:
model.eval()
for image_id, _ in test_loader:
  

In [0]:
labels = []
model.eval()
for i, image_id in enumerate(test_df['image_names']):
  img_name = DIR_IMAGES + image_id
  image = cv2.imread(img_name, cv2.IMREAD_COLOR)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  transformed = transforms_valid(image=image)
  image = transformed['image']
  image_tensor = image.unsqueeze_(0)
  input = torch.autograd.Variable(image_tensor)
  input = input.to(device)
  output = model(input)
  index = output.data.cpu().numpy().argmax()
  labels.append(index)

KeyboardInterrupt: ignored

In [0]:
labels

[0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
